In [2]:
import pandas as pd
import torchaudio.transforms as T
from pytube import YouTube
import torchaudio
from tqdm.notebook import tqdm
import numpy as np
import glob
from sklearn.model_selection import StratifiedKFold
from pytube.exceptions import VideoPrivate

In [26]:
data = pd.read_csv('wanted.csv')

In [27]:
data

,YTID,start,end,labels
0,-9ek6eO0RtI,260.0,270.0,/m/03wwcy
1,-JKLmqDk9p8,490.0,500.0,"/m/06mb1,/m/07r10fb,/m/0jb2l,/m/0ngt1,/t/dd00038"
2,-gStrDahsjU,20.0,30.0,"/m/03wwcy,/m/04rlf,/m/09x0r,/t/dd00126"
3,-uSlzn9oPQ4,20.0,30.0,"/m/06mb1,/m/0jb2l,/m/0ngt1,/t/dd00038"
4,021B2ozAjNg,110.0,120.0,"/m/01h8n0,/m/09x0r"
...,...,...,...,...
235,yw9MbiKIhdM,120.0,130.0,"/m/01h8n0,/m/09x0r"
236,zQpPfONDPXs,360.0,370.0,"/m/0463cq4,/m/09x0r,/t/dd00002,/t/dd00125"
237,zY1LJu1j4zw,30.0,40.0,/t/dd00002
238,zkNbNA2mLIg,20.0,30.0,/t/dd00002


In [28]:
class_map = {
    '/t/dd00002':'Baby',
    '/m/03wwcy':'Doorbell',
    '/m/01h8n0':'Conversation',
    '/t/dd00038':'Rain'
}

In [36]:
def download_audio(df:pd.DataFrame):
    miss_vids = []
    root_url = "https://www.youtube.com/watch?v="
    n = len(df)
    for i in tqdm(range(n)):
        uid = df.loc[i,'YTID']
        l = df.loc[i,'labels'].split(',')
        if len(l) == 1:
            p = class_map[l[0]]
        else:
            for j in class_map.keys():
                if j in l:
                    p = class_map[j]
        try:
            yt = YouTube(url=root_url+uid)
            title = yt.title
            stream = yt.streams.filter(only_audio=True)
            print(stream[1])
            miss_vids.append({'path':f"{p}/{str(i)}",'vides-id':uid,'start_time':df.loc[i,'start'],'end_time':df.loc[i,'end'],'class':p})
            print(yt)
        except (Exception):
            print(f"{uid}")
            pass
        
        else:
            stream[1].download(p,filename=str(i)+'.mp4')
    return miss_vids        

In [37]:
missed = download_audio(data)

  0%|          | 0/240 [00:00<?, ?it/s]

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
<pytube.__main__.YouTube object: videoId=-9ek6eO0RtI>
<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
<pytube.__main__.YouTube object: videoId=-JKLmqDk9p8>
-gStrDahsjU
<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
<pytube.__main__.YouTube object: videoId=-uSlzn9oPQ4>
<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
<pytube.__main__.YouTube object: videoId=021B2ozAjNg>
<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
<pytube.__main__.YouTube object: videoId=02JEvDAcq7U>
<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
<pytube.__main__.YouTube object: videoId=09w-ZFasRvE>
<Stream: i

In [38]:
df = pd.DataFrame(missed)

In [39]:
df.to_csv("160.csv",index=False)

## Analysis and Preprocessing

In [5]:
c = {
    'Baby': 0,
    'Conversation':1,
    'Doorbell':2,
    'Rain':3
}

In [6]:
aud_df = pd.read_csv("160.csv")

In [11]:
aud_df['class'] = aud_df['class'].apply(lambda x: c[x])

In [15]:
aud_df['duration'] = aud_df['end_time'] - aud_df['start_time']

In [16]:
aud_df.to_csv("160.csv",index=False)

In [14]:
torchaudio.load(aud_df.loc[0,'path']+'.wav')

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 44100)

In [69]:
aud_df['folds'] = -1
for i,(train_ind,val_ind) in enumerate(StratifiedKFold(n_splits=5).split(aud_df['path'],aud_df['class'])):
    aud_df.loc[val_ind,'folds'] = i


In [68]:
np.random.randint(400)

317

In [71]:
aud_df.to_csv('160.csv',index=False)

In [73]:
import torch

In [96]:
s = torch.zeros((2,128,3446),dtype=torch.half)

In [98]:
s

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float16)